<a href="https://colab.research.google.com/github/Aba17/NLP_Word_vectors/blob/main/Ababacar_Diokhane%CC%81_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [ ]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [ ]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [ ]:

print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")

load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [ ]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []
    for sentence in data:
        sentence_with_unk=[]
        for word in sentence:
            if word not in vocab or vocab[word]<=mincount:
                sentence_with_unk.append("<unk>")
            else:
                sentence_with_unk.append(word)
        data_with_unk.append(sentence_with_unk)

    ## FILL CODE
    
    return data_with_unk

In [ ]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [ ]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        # print(sentence)
        # break
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        # if sentence in vocab:
        for i in range(len(sentence)):
            if i+n<len(sentence):
                context=sentence[i:i+n]
                word=sentence[i+n]
                if word not in counts[context].keys():
                    
                    counts[context][word]=1
                else:
                    counts[context][word]+=1
            
            #Add one word
            if (i+1)<len(sentence):
                one_context=sentence[i]
                word=sentence[i+1]
                if word not in counts[one_context]:
                    counts[one_context][word]=1
                else:
                    counts[one_context][word]+=1

                
            


    

                           

    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!
    ponctuations=['.',',',':','?','!',';']


    for context in counts.keys():
        total_number_words=0
        for word in counts[context]:
            total_number_words+= counts[context][word]
        for word in counts[context]:
            good_word=word
            for ponctuation in ponctuations:
                if ponctuation in word and len(word)>1:
                    good_word=word.replace(ponctuation,'')
            if word in prob[context]:
                prob[context][word]+=counts[context][word]/total_number_words
            else:
                prob[context][word]=counts[context][word]/total_number_words

    ## FILL CODE

    return prob

In [ ]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model= build_ngram(train_data, n)


build ngram model with n =  2


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [ ]:
# el,max_el=None,0
# for cont in model:
#     for word in model[cont]:
#         if model[cont][word]>max_el:
#             el=model[cont]
#             max_el=model[cont][word]

# print(max_el)
# counts[('decision', 'to')]# aFFICHER CE LA POUR COMPRENDRE LA NORMALISATION

In [ ]:
# model[('to')]

In [ ]:
# el,max_el=None,0

# for word in counts[('that')]:
#     if counts[('that')][word]>max_el:
#         el=word
#         max_el=counts[('that')][word]
# el,max_el

In [ ]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    # best_prob=0.0

    # print("Context:" , context)
    if context in model:
        prob=model[context][w]
        context=context[1:]
        if type(context) == tuple and len(context)==1:
            context=context[0]
    else:
        prob=0
        context=context[1:]
        if type(context) == tuple and len(context)==1:
            context=context[0]
    # print("Context hgfg:" , context)
    
    if context :
        return np.max([prob,get_prob(model,context,w)])
    else :
        return 0
   

        



In [ ]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of th== word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    best_prob=0.0
    perplexity=0
    
    for sentence in data:
        sentence_perplexity=1.0
        size_sentence=len(sentence)
        
        sentence=tuple(sentence)
        # print("Taille : ",size_sentence)
        for i in range(size_sentence):
            if i+n<len(sentence):
                
                context=sentence[i:i+n]
                best_prob=0
                best_word=""

                # print("On entre : ",context)
                while context:
                    # print("Begining ",context)
                    if not model[context]:
                        # print("Not Good ",context)
                        context=context[1:]
                        if type(context) == tuple and len (context)==1:
                            context=context[0]
                        
                        # print("New ",context)
                    else:
                        break
                
                if context:
                    for word in model[context]:
                        prob=get_prob(model, context, word)
                        
                        if prob>best_prob :
                            # and word!='<unk>'
                            best_prob=prob
                            best_word=word
                

                # print(" best eword : ",best_word)
                if best_prob==0:#For element that are not in the model
                    best_prob=1
                sentence_perplexity*=best_prob
        print("Prob : ",best_prob,"sent : ",sentence_perplexity," Contexte  : ",context)
                # break
                
        T=-(1/size_sentence)
        # print('T :',T)
        sentence_perplexity=math.pow(sentence_perplexity,T)
        print("Perp : ",sentence_perplexity)
        perplexity+=sentence_perplexity


    return perplexity

In [ ]:
# COMPUTE PERPLEXITY ON VALIDATION SET
print("The perplexity is", perplexity(model, valid_data, n=n))
# 

Prob :  1.0 sent :  0.00021673181778775977  Contexte  :  ('do', 'that.')
Perp :  2.3249184964768994
Prob :  1.0 sent :  0.0023421618595881335  Contexte  :  ('the', 'situation.')
Perp :  1.9600393409308574
Prob :  1.0 sent :  0.00236612702366127  Contexte  :  that.
Perp :  2.3721241807801055
Prob :  1.0 sent :  0.0001267876147529549  Contexte  :  ('a', 'week.')
Perp :  2.260818730549134
Prob :  0.26335078534031414 sent :  0.03467681806968011  Contexte  :  ('<unk>', '<unk>')
Perp :  1.7511638903605713
Prob :  0.26335078534031414 sent :  4.056355675449051e-05  Contexte  :  <unk>
Perp :  2.322676170410066
Prob :  1.0 sent :  8.716444382669764e-08  Contexte  :  ('do', 'that?')
Perp :  2.168574347089617
Prob :  1.0 sent :  7.598557622437027e-06  Contexte  :  ('money', '<unk>')
Perp :  2.47622389814005
Prob :  1.0 sent :  0.02693602693602693  Contexte  :  ('children', '<unk>')
Perp :  1.8264638337109151
Prob :  0.34424778761061947 sent :  0.00041497131682579036  Contexte  :  ('the', '<unk>')


In [ ]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    # words_and_probs = defaultdict(lambda: 0.0)

    # print("Context : ",context)
    if context not in model[context] and type(context) == tuple and len (context)==1:
        context=context[0]
    if not context:
        # print("1")
        return None
    else:
        u_context=context[1:]
        if type(u_context) == tuple and len (u_context)==1:
            u_context=u_context[0]
        words_and_probs= defaultdict(lambda: 0.0)
        if len(context)==1:
            context==context[0]
        if context in model:
            for word in model[context]:
                words_and_probs[word]=model[context][word]
        
        u_words_and_probs=get_proba_distrib(model, u_context)
        if u_words_and_probs:
            for word in u_words_and_probs:
                if words_and_probs[word]:
                    words_and_probs[word]+=model[context][word]
                else:
                    words_and_probs[word]=model[context][word]
        # print("2")
        return words_and_probs

In [ ]:
# model[('to','do')]

In [ ]:
# model["<s>"]

In [ ]:
# sentence = ['tom', 'and']
# proba = get_proba_distrib(model, tuple(sentence))
# proba

In [ ]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        context=tuple(sentence)
        if len(sentence)>n:
            context=tuple(sentence[-n:])
        # print(" entence :",sentence,"Context : ",context)
        distrib= get_proba_distrib(model, context)
        word_of_max_proba=""
        max_proba=0
        for word in distrib:
            if word=="<unk>":
                continue
            if distrib[word]>max_proba:
                max_proba=distrib[word]
                word_of_max_proba=word
        
        # print("Pred : ",max_proba," kjhj : ",word_of_max_proba)
        
        sentence.append(word_of_max_proba)
    
    return sentence

In [ ]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'tom', 'and', 'mary', 'said', 'they', 'were', 'going', 'to', 'be', 'the', 'first', 'time', 'that', "you've", 'asked', 'tom', 'to', 'the', 'station.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

In [ ]:
train_data_2,vocab=load_data("train2.txt")
valid_data_2,_=load_data("valid2.txt")
train_data_2 = remove_rare_words(train_data_2, vocab, mincount = 1)
valid_data_2 = remove_rare_words(valid_data_2, vocab, mincount = 1)


In [ ]:
train_data_2[0]

['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']

In [ ]:
model2=build_ngram(train_data_2, 2)


In [ ]:
# model2

In [ ]:

print("Generated sentence: ",generate(model2))

Generated sentence:  ['<s>', 'i', "'m", 'not', 'going', 'to', 'the', 'station', '.', '</s>']
